# 작업형2

## [가격 예측] 중고 자동차
- 자동차 가격을 예측해주세요!


- 예측할 값(y): price
- 평가: RMSE (Root Mean Squared Error)
- data: train.csv, test.csv
- 제출 형식: result.csv파일을 아래와 같은 형식(수치형)으로 제출
~~~
pred
11000
20500
19610
...
11995
~~~

### 답안 제출 참고
- pd.read_csv('result.csv') 로 제출 코드 확인

In [52]:
# 데이터 불러오기
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
# print(train.shape, test.shape)
# (3759, 9) (1617, 8)

# print(train.head())
#        model  year  price transmission  mileage fuelType  tax   mpg  \
# 0   EcoSport  2017  10298       Manual    25013   Petrol  150  53.3   
# 1      Focus  2016  10491       Manual    30970   Diesel    0  74.3   
# 2      S-MAX  2017  13498       Manual    60200   Diesel  150  56.5   
# 3     Fiesta  2018  10600       Manual    27380   Petrol  145  56.5   
# 4     Fiesta  2018  11000       Manual     7724   Petrol  145  65.7   

#    engineSize  
# 0         1.0  
# 1         1.5  
# 2         2.0  
# 3         1.0  
# 4         1.0  
# print(test.head())
#      model  year transmission  mileage fuelType  tax   mpg  engineSize
# 0    S-MAX  2016       Manual    23905   Diesel  125  56.5         2.0
# 1   Fiesta  2018       Manual    16895   Petrol  145  40.3         1.5
# 2     Kuga  2017       Manual    12109   Petrol  165  45.6         1.5
# 3     Kuga  2018    Semi-Auto    13940   Diesel  145  58.9         1.5
# 4   Fiesta  2013       Manual    31690   Petrol   30  54.3         1.2

# print(train.describe())
#               year         price        mileage          tax          mpg  \
# count  3759.000000   3759.000000    3759.000000  3759.000000  3759.000000   
# mean   2016.887736  12353.321362   23191.786379   114.444001    57.793083   
# std       2.151811   4750.338051   19223.258713    61.713682    10.099221   
# min    2002.000000   1095.000000       5.000000     0.000000    20.900000   
# 25%    2016.000000   9000.000000    9997.500000    75.000000    52.300000   
# 50%    2017.000000  11495.000000   18180.000000   145.000000    58.900000   
# 75%    2018.000000  15310.000000   31000.000000   145.000000    65.700000   
# max    2060.000000  48000.000000  144000.000000   570.000000   201.800000   

#         engineSize  
# count  3759.000000  
# mean      1.352807  
# std       0.437130  
# min       0.000000  
# 25%       1.000000  
# 50%       1.200000  
# 75%       1.500000  
# max       5.000000  

# print(train.describe(include="O"))
#           model transmission fuelType
# count      3759         3759     3759
# unique       19            3        3
# top      Fiesta       Manual   Petrol
# freq       1372         3234     2541
# print(test.describe(include="O"))
#           model transmission fuelType
# count      1617         1617     1617
# unique       19            3        3
# top      Fiesta       Manual   Petrol
# freq        591         1385     1085

# print(test.isnull().sum()) # 없음

# 0. 타겟값 분리
target = train.pop('price')
# print(train.shape, test.shape) #(3759, 8) (1617, 8)
# print(target)

# 1. base라인 RMSE:  1543.7584595763765
n_cols = train.select_dtypes(exclude="O").columns

# 2. 문자열 변환 + 인코딩 + 정규화 RMSE:  1319.9210856805942
o_cols = train.select_dtypes(include="O").columns

from sklearn.preprocessing import LabelEncoder, StandardScaler

for col in o_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

scaler = StandardScaler()
train[n_cols] = scaler.fit_transform(train[n_cols])
test[n_cols] = scaler.transform(test[n_cols])

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=2022)


from sklearn.metrics import mean_squared_error
def rmse(y_val, pred):
    return(mean_squared_error(y_val, pred)**0.5)

# 3. 모델 선택
# RF: RMSE:  1319.9210856805942
# XGB: RMSE:  1269.7677864640748
# light: RMSE:  1263.5790505420978 

# from sklearn.ensemble import RandomForestRegressor
# model = RandomForestRegressor(random_state=2022)
# from xgboost import XGBRegressor
# model = XGBRegressor(random_state=2022)

# 4. 하이퍼 파라미터 튜닝: n_estimators = 200, RMSE:  1244.2510087245782
from lightgbm import LGBMRegressor
model = LGBMRegressor(n_estimators = 200, random_state=2022, verbose=-1)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)
print("RMSE: ", rmse(y_val, pred))
pred = model.predict(test)
submit = pd.DataFrame({
    'pred': pred
})
submit.to_csv('작업형2제출.csv', index=False)

RMSE:  1244.2510087245782


In [53]:
y = pd.read_csv("y.csv")
pred = pd.read_csv('작업형2제출.csv')
rmse(y, pred)

1365.2827536351276